In [4]:
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import shutil


import os
import IPython.display

In [5]:
def load(file_name):
    wav = file_name
    file_sr = librosa.get_samplerate(wav)

    y, sr = librosa.load(wav, sr=file_sr)
    return y, sr

In [3]:
def checkpoint(y, mininum):
    start = []
    end = []
    first = True
    con = 0
    
    for time in range (len(y)):
        if(abs(y[time]) > mininum):
            if(first):
                first = False
                start_time = time
            else:
                end_time = time
        else:
            con = con + 1
            if(con > 20000):
                con = 0
                if(not first) :
                    start.append(start_time)
                    end.append(end_time)
                    first = True
    return start, end

In [4]:
def crop(y, sr, start, end, left_padding, right_padding, outputdir):
    for i in range(len(start)):
        height = (end[i]-start[i])/3000
        width = (end[i]-start[i])/500
        if(height > 4) :
            fig = plt.figure(figsize=(width, height))
            sf.write(outputdir + str(i) + ".wav", y[start[i]+left_padding:end[i]+right_padding], sr, subtype='PCM_24')

In [3]:
def make_background(y, sr, outputdir, startpoint=1):
    during = 2 * sr
    amount = int(len(y) / during)
    
    for i in range(amount):
        start = (i*2) * sr
        end = start + during
        newy = y[start:end]
        resample = librosa.resample(newy, sr, 16000)
        sf.write(outputdir + "bg_" + str(i+startpoint) + ".wav", resample, 16000, subtype='PCM_16')

In [17]:
def split_noise(y, sr, outputdir, name):
    during = 2 * sr
    amount = int(len(y) / during)
    
    for i in range(amount):
        start = (i*2) * sr
        end = start + during
        newy = y[start:end]
        resample = librosa.resample(newy, sr, 16000)
        sf.write(outputdir + name + str(i) + ".wav", resample, 16000, subtype='PCM_16')

In [2]:
def book_train(y, sr, outputdir, name):
    
    resample = librosa.resample(y, sr, 16000)
    
    for i in range (int(len(resample)/16000)-1):
        
        p = i * 16000
        q = p + 32000
        split = resample[p:q]
    
#         max_mine = np.max(split)
#         ratio = 0.46202388 / max_mine
    
#         split = split * ratio
        
        sf.write(outputdir + name + str(i) + ".wav", split, 16000, subtype='PCM_16')

In [15]:
def make_delete_list(input_directory):

    file_list = os.listdir(input_directory)
    delete_list = []
    for file in file_list:
        y, sr = load(input_directory+file)
        sum = 0
        for i in range(sr):
            sum = sum + abs(y[i])
        if(sum < 100):
            delete_list.append(file)
    return delete_list

In [6]:
y, sr = load('original/fire/121.wav')
start, end = checkpoint(y, 0.05)
print(len(start), len(end))

2 2


In [7]:
crop(y, sr, start, end, 0, 9600, 'split/fire/')

<Figure size 2374.85x395.808 with 0 Axes>

<Figure size 5669.28x944.88 with 0 Axes>

In [33]:
y, sr = load('the_little_prince/1/the_little_prince_dh2.wav')
sf.write('the_little_prince/1/the_little_prince_dh2.wav', y[:-sr*3], sr, subtype='PCM_16')

In [6]:
filelist = os.listdir('the_little_prince/original')

for file in filelist:
    noise, noise_sr = librosa.load('the_little_prince/original/' + file)
    book_train(noise, noise_sr, 'the_little_prince/split_new/', file)

In [12]:
delete_list = make_delete_list('the_little_prince/split_new/')

In [13]:
len(delete_list)

233

In [14]:
for file in delete_list:
    shutil.move('the_little_prince/split_new/'+file, 'the_little_prince/del/'+file)

In [ ]:
for i in range(8):
    y, sr = load('background/'+ str(i+1) + ".wav")
    make_background(y, sr, 'background/audiobook/', (i+1)*10000)

In [22]:
delete_list = make_delete_list('background/bg/')
for file in delete_list:
    shutil.move('background/bg/'+file, 'background/background_delete/'+file)

In [10]:
sf.write("split/fire/5.wav", y[start[1]:end[2]], sr, subtype='PCM_24')

In [ ]:
y, sr = load('background/background_delete/bg_807.wav')
sum = 0
for i in range(sr):
    sum = sum + abs(y[i])
print(sum)

In [ ]:
librosa.display.waveplot(y, sr=sr)

In [7]:
y, sr = load('dj28.wav')

In [8]:
sr

16000

In [8]:
y = y[10000000:-1100000]
sf.write("200318.wav", y, sr, subtype='PCM_16')

In [ ]:
IPython.display.Audio(data=y[10000000:11000000], rate=sr)

In [ ]:
IPython.display.Audio(data=y[-1100000:], rate=sr)

In [ ]:
len